In [ ]:
# checking for tensorflow devices
from tensorflow.python.client import device_lib
for d in device_lib.list_local_devices():
    print(d.name, d.physical_device_desc)

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras import optimizers
import keras.backend as K
from keras.utils import to_categorical, multi_gpu_model
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.image import ImageDataGenerator
from sklearn.datasets import load_iris
from keras.datasets import fashion_mnist, mnist, cifar100
import tensorflow as tf

np.set_printoptions(suppress=True)
%matplotlib inline

In [ ]:
# dataset generators for test and train
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# dataset generators for test and train
train_generator = train_datagen.flow_from_directory(
        'kaggle/dogs-vs-cats/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        'kaggle/dogs-vs-cats/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

In [ ]:
# all images are scaled to the same size
input_shape = (150, 150, 3)

In [ ]:
# cnn model
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape, name='L0'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64, (3,3), activation='relu', name='L1'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(128, (3,3), activation='relu', name='L2'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())         
model.add(Dense(512, activation='relu')) 
model.add(Dropout(0.2))                  
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# making the sequential model run across multiple GPUs
opt = optimizers.SGD(lr=0.001)

parallel_model = multi_gpu_model(model, 4)
parallel_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
parallel_model.summary()

In [ ]:
# number of images per batch, and total images
train_generator.batch_size, train_generator.samples

In [ ]:
# number of batches in epoch
# added 2x for training to get more random samples
train_batches_per_epoch = int(train_generator.samples / train_generator.batch_size) * 2
validation_batches_per_epoch = int(validation_generator.samples / validation_generator.batch_size)
train_batches_per_epoch, validation_batches_per_epoch

In [ ]:
# get a batch and its classes
batch = train_generator.next()
batch[1]

In [ ]:
# there are 32, 150x150, 3-channel images in this batch
batch[0].shape

In [ ]:
# show one sample from the batch
plt.imshow(batch[0][0]);

In [ ]:
history = parallel_model.fit_generator(
        train_generator,
        steps_per_epoch=train_batches_per_epoch,
        epochs=25,
        validation_data=validation_generator,
        validation_steps=validation_batches_per_epoch,
        workers=32,
        use_multiprocessing=True)